In [215]:
import os,sys,getopt
import numpy as np
import netCDF4

In [216]:
possible_nodal_attributes=['primitive_weighting_in_continuity_equation',
                           'surface_submergence_state',
                           'quadratic_friction_coefficient_at_sea_floor',
                           'surface_directional_effective_roughness_length',
                           'surface_canopy_coefficient',
                           'bridge_pilings_friction_paramenters',
                           'mannings_n_at_sea_floor',
                           'chezy_friction_coefficient_at_sea_floor',
                           'sea_surface_height_above_geoid',
                           'bottom_roughness_length',
                           'wave_refraction_in_swan',
                           'average_horizontal_eddy_viscosity_in_sea_water_wrt_depth',
                           'elemental_slope_limiter']

In [273]:
#
nc=netCDF4.Dataset('maxele.63.nc')
nopen=nc['max_nvdll'][0]
print "Grid has {} open boundary nodes.".format(nopen)
ibtype=nc['ibtype'][:]
fort20=False
if any(ibtype==22) : 
    fort20=True
    print "Grid has river nodes."

nbvv=nc['nbvv']
print nbvv.shape

Grid has 119 open boundary nodes.
Grid has river nodes.
(34654, 241)


In [270]:
#

f='fort.15'
if not os.path.isfile(f):
    print f + " not found."
    sys.exit()
    
f15 = open(f, 'r')
header=f15.readline()
ident=f15.readline() 
nfover=f15.readline().split()[0]
nabout=f15.readline().split()[0]
nscreen=f15.readline().split()[0]
ihot=f15.readline().split()[0]
ics=f15.readline().split()[0]
im=f15.readline().split()[0]
nolibf=f15.readline().split()[0]
nolifa=f15.readline().split()[0]
nolica=f15.readline().split()[0]
nolicat=f15.readline().split()[0]
nwp=f15.readline().split()[0]
for i in range(int(nwp)):
    nwp_val=f15.readline().split()[0]
    if not nwp_val in possible_nodal_attributes :
        print "Nodal attribute {} not known.".format(nwp_val)
        sys.exit()

ncor=f15.readline().split()[0]
ntip=int(f15.readline().split()[0])
nws=f15.readline().split()[0]
nramp=f15.readline().split()[0]
g=f15.readline().split()[0]
tau=f15.readline().split()[0]
# check for taumin,,,

dt=f15.readline().split()[0]
statim=f15.readline().split()[0]
reftim=f15.readline().split()[0]
if int(nws)>0:
    wtiminc=f15.readline().split()
    
    
rnday=float(f15.readline().split()[0])
dramp=f15.readline().split()

timeweights=f15.readline().split()[0:3];
timeweights = map(float, timeweights)

if np.sum(timeweights) > 1 :
    print "Time weights sum > 1.  this is dumb." 
    sys.exit()
elif im == '111112' :
    if not timeweights[1] == 1 :
        print "Middle time weight should be 1.0 if IM=='111112'."
        sys.exit()

h0=f15.readline().split()

SLAM0,SFEA0=f15.readline().split()[0:2];

ffactor=f15.readline().split()
esl=f15.readline().split()[0]
cori=f15.readline().split()[0]

# process tidal potential forcing
ntif=int(f15.readline().split()[0])
print "Number of tidal potential forcing frequencies = {}".format(ntif)
if ntip == 0 and ntif>0 :
    print "Tidal potential forcing will not be used, even though it is specified (NTIF>0)." 
for i in range(ntif) :
    cname=f15.readline().split()[0]
    cname_facs=f15.readline().split()
    #print "{}: Scanned {}:{} ".format(i,cname,cname_facs)

# process open boundary forcing
nbfr=int(f15.readline().split()[0])
print "Number of boundary forcing frequencies = {}".format(nbfr)
for i in range(nbfr) :
    cname=f15.readline().split()[0]
    #print "{}: Scanning {} ".format(i,cname)
    cname_facs=f15.readline().split()
for i in range(nbfr) :
    cname=f15.readline().split()
    #print "{}: Constituent {} ".format(i,cname)
    for j in range(nopen) :
        amppha=f15.readline().split()
        #print "     {}: Scanning {} ".format(j,amppha)   
    
ANGINN=f15.readline().split()
if fort20 : 
    NFFR=int(f15.readline().split()[0])
    if NFFR == 0 or NFFR == -1 :
        if not os.path.isfile('fort.20'):
            print "fort.20 not found. This will be terminal at runtime."
        else:
            f20=open('fort.20', 'r')
            f20_timeinc=float(f20.readline().split()[0])
            

NOUTE_Line=f15.readline().split()
NNOUTE=int(f15.readline().split()[0])
for i in range(NNOUTE) :
    line=f15.readline().split()

NOUTV_Line=f15.readline().split()
NNOUTV=int(f15.readline().split()[0])
for i in range(NNOUTV) :
    line=f15.readline().split()

if nws > 0 :
    NOUTM_Line=f15.readline().split()
    NNOUTM=int(f15.readline().split()[0])
    for i in range(NNOUTM) :
        line=f15.readline().split()    

NOUTGE_Line=f15.readline().split()
NOUTGV_Line=f15.readline().split()
if nws > 0 :
    NOUTGM_Line=f15.readline().split()


NHARF=int(f15.readline().split("!")[0])
THAS, THAF, NHAINC,FMV=f15.readline().split()[0:4]

NHASE,NHASV,NHAGE,NHAGV=f15.readline().split()[0:4]
NHSTAR,NHSINC=f15.readline().split()[0:2]
   
ITITER,ISLDIA,CONVCR,ITMAX=f15.readline().split()[0:4]

f15.close()      

# check values/files
ihot=int(ihot)
print "ihot={}".format(ihot)
if ihot == 67 : 
    if not os.path.isfile('fort.67'):
        print "ihot == {}, but fort.67 not found. This will be terminal at runtime.".format(ihot)
elif ihot == 68 : 
    if not os.path.isfile('fort.68'):
        print "ihot == {}, but fort.68 not found. This will be terminal at runtime.".format(ihot)
elif ihot == 567 : 
    if not os.path.isfile('fort.67.nc'):
        print "ihot == {}, but fort.67.nc not found. This will be terminal at runtime.".format(ihot)
elif ihot == 568 : 
    if not os.path.isfile('fort.68.nc'):
        print "ihot == {}, but fort.68.nc not found. This will be terminal at runtime.".format(ihot)
    nc=netCDF4.Dataset('fort.68.nc')
    rndayInHs=nc.rnday
    print "rnday={}, rnday2={}".format(rnday,rnday2)
    if rnday > rndayInHs : 
        print "   RNDAY exceeds time in hotstart file.  This is good." 
    elif rnday == rndayInHs : 
        print "   RNDAY equals time in hotstart file.  This is dumb.  Nothing will get computed." 
    elif rnday > rndayInHs : 
        print "   RNDAY less than time in hotstart file.  This is bad." 



Number of tidal potential forcing frequencies = 8
Number of boundary forcing frequencies = 8
ihot=568
rnday=69.25, rnday2=45.0
   RNDAY exceeds time in hotstart file.  This is good.


In [171]:
pwd

u'/Users/bblanton/Sites/NoteBooks'